## Analytic Assignment 1

### Phase 1: Supercell vs. Competition

In the first phase of the analysis we are looking to understand  how is Supercell doing vs. King and Zynga. By answering the below questions, we would set the scene on which we can build the analysis further:

a)  What is the average monthly  user growth  rates of Supercell, King and Zynga?

b)  What is the net reach of Supercell, King and Zynga in the sample for each of the 3 months?

c)  What is the total time spent per month  per user for Supercell, King and Zynga for July?

d)  Which of Supercell, King and Zynga had the largest average session duration in June?

e)  What is the gender distribution of Supercell vs. King users?

In [1]:
import os.path
import datetime
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
data_path = os.path.join(os.path.dirname('__file__'), "Verto_Data_Analyst_Assignment/Assignment_1_Data/")

In [3]:
demographics_df = pd.read_csv(data_path + "demographics_data.csv", sep=';')
devices_df = pd.read_csv(data_path + "devices_data.csv", sep=';')
dictionary_df = pd.read_csv(data_path + "dictionary_data.csv", sep=';')
main_branches_df = pd.read_csv(data_path + "main_branches.csv", sep=';')
sessions_df = pd.read_csv(data_path + "sessions_data.csv", sep=';')
subjects_tree_df = pd.read_csv(data_path + "subjects_tree_data.csv", sep=';')

#### a)  What is the average monthly  user growth  rates of Supercell, King and Zynga?

In [4]:
subjects_id_sessions_df = sessions_df[~sessions_df['subject_id'].isin(subjects_tree_df['branch_id'])]
subjects_id_sessions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141750 entries, 0 to 141926
Data columns (total 7 columns):
subject_id          141750 non-null int64
start_timestamp     141750 non-null object
end_timestamp       141750 non-null object
interaction_type    141750 non-null object
user_id             141750 non-null int64
device_id           141750 non-null int64
os                  141750 non-null object
dtypes: int64(3), object(4)
memory usage: 8.7+ MB


In [5]:
branches_id_sessions_df = sessions_df[sessions_df['subject_id'].isin(subjects_tree_df['branch_id'])]
branches_id_sessions_df = branches_id_sessions_df.rename(columns = {'subject_id':'branch_id'})

id_modification_df = pd.merge(branches_id_sessions_df, subjects_tree_df, on='branch_id', how='left')
id_modification_df.drop(['readable_name', 'parent_branch_id', 'ancestors_branch_ids'],inplace=True,axis=1)
id_modification_df = id_modification_df.drop_duplicates(subset =['branch_id', 'start_timestamp'], keep='last')
id_modification_df.drop(['branch_id'],inplace=True,axis=1)

In [6]:
user_growth_rates_df = pd.concat([subjects_id_sessions_df, id_modification_df])
user_growth_rates_df.reset_index(inplace=True, drop=True)
user_growth_rates_df.head()

,device_id,end_timestamp,interaction_type,os,start_timestamp,subject_id,user_id
0,10212,2016-07-02 05:35:36,app,android,2016-07-02 05:29:24,5245226,2694
1,10212,2016-07-19 13:33:20,app,android,2016-07-19 13:30:27,5245226,2694
2,10212,2016-06-26 19:13:03,app,android,2016-06-26 19:10:29,5245226,2694
3,10212,2016-05-18 02:03:57,app,android,2016-05-18 01:51:42,5245226,2694
4,10212,2016-06-23 05:06:51,app,android,2016-06-23 05:02:43,5245226,2694


In [7]:
main_branches_df['property'] = main_branches_df['property'].str.lower()
dictionary_df = dictionary_df.rename(columns = {'readable_name':'property'})
dictionary_df['property'] = dictionary_df['property'].str.lower()
dictionary_df = pd.merge(dictionary_df, main_branches_df, on='property', how='left')
dictionary_df.drop(['branch_id'],inplace=True,axis=1)

dictionary_df.ix[dictionary_df['property'].str.startswith('facebook'), 'company_name'] = "Facebook"
dictionary_df.ix[dictionary_df['property'].str.startswith('fb'), 'company_name'] = "Facebook"
dictionary_df.ix[dictionary_df['property'].str.startswith('amazon'), 'company_name'] = "Amazon"
dictionary_df.ix[dictionary_df['property'].str.startswith('youtube'), 'company_name'] = "Google"
dictionary_df.ix[dictionary_df['property'].str.startswith('walmart'), 'company_name'] = "Walmart"
dictionary_df.ix[dictionary_df['property'].str.startswith('wal-mart'), 'company_name'] = "Walmart"
dictionary_df.ix[dictionary_df['property'].str.startswith('words with friends'), 'company_name'] = "Zynga"
dictionary_df.ix[dictionary_df['property'].str.startswith('lovefilm'), 'company_name'] = "Lovefilm"

In [8]:
user_growth_rates_df = pd.merge(user_growth_rates_df, dictionary_df, on='subject_id', how='left')
user_growth_rates_df.head()

,device_id,end_timestamp,interaction_type,os,start_timestamp,subject_id,user_id,property,company_name
0,10212,2016-07-02 05:35:36,app,android,2016-07-02 05:29:24,5245226,2694,youtube,Google
1,10212,2016-07-19 13:33:20,app,android,2016-07-19 13:30:27,5245226,2694,youtube,Google
2,10212,2016-06-26 19:13:03,app,android,2016-06-26 19:10:29,5245226,2694,youtube,Google
3,10212,2016-05-18 02:03:57,app,android,2016-05-18 01:51:42,5245226,2694,youtube,Google
4,10212,2016-06-23 05:06:51,app,android,2016-06-23 05:02:43,5245226,2694,youtube,Google


In [9]:
user_growth_rates_df['start_timestamp'] = pd.to_datetime(user_growth_rates_df['start_timestamp'])
user_growth_rates_df['end_timestamp'] = pd.to_datetime(user_growth_rates_df['end_timestamp'])
user_growth_rates_df['month'] = user_growth_rates_df['start_timestamp'].map(lambda x: x.month)
user_growth_rates_df.head()

,device_id,end_timestamp,interaction_type,os,start_timestamp,subject_id,user_id,property,company_name,month
0,10212,2016-07-02 05:35:36,app,android,2016-07-02 05:29:24,5245226,2694,youtube,Google,7
1,10212,2016-07-19 13:33:20,app,android,2016-07-19 13:30:27,5245226,2694,youtube,Google,7
2,10212,2016-06-26 19:13:03,app,android,2016-06-26 19:10:29,5245226,2694,youtube,Google,6
3,10212,2016-05-18 02:03:57,app,android,2016-05-18 01:51:42,5245226,2694,youtube,Google,5
4,10212,2016-06-23 05:06:51,app,android,2016-06-23 05:02:43,5245226,2694,youtube,Google,6


In [10]:
user_growth_rates_df.groupby(['company_name', 'month']).user_id.nunique()

company_name  month
Amazon        5         89
              6         79
              7         86
Facebook      5        100
              6        100
              7         99
Google        5         97
              6         95
              7         95
King          5         33
              6         36
              7         36
Lovefilm      5          5
              6          2
              7          6
Supercell     5         19
              6         17
              7         18
Walmart       5         63
              6         57
              7         54
Zynga         5         28
              6         30
              7         29
Name: user_id, dtype: int64

In [11]:
# (users number in July) = (users number in May) * (1 + monthly growth rate)^2
# (monthly growth rate) = sqrt((users number in July) / (users number in May)) - 1

King_growth_rate = math.sqrt(36/33) - 1
Supercell_growth_rate = math.sqrt(18/19) - 1
Zynga_growth_rate = math.sqrt(29/28) - 1

print('Average monthly user growth rates of Supercell: ', Supercell_growth_rate)
print('Average monthly user growth rates of King: ', King_growth_rate)
print('Average monthly user growth rates of Zynga: ', Zynga_growth_rate)

Average monthly user growth rates of Supercell:  -0.026671473215424846
Average monthly user growth rates of King:  0.044465935734187
Average monthly user growth rates of Zynga:  0.017700489198214875


#### b) What is the net reach of Supercell, King and Zynga in the sample for each of the 3 months?

In [12]:
# net reach for King
king_df = user_growth_rates_df[user_growth_rates_df['company_name']=='King']
king_df = pd.DataFrame(king_df.groupby(['month']).user_id.unique())
king_net_reach_may = 33
king_net_reach_june = king_net_reach_may + 36 - len(set(king_df.ix[5][0]) & set(king_df.ix[6][0]))
king_net_reach_july = king_net_reach_june + 36 - len(set(king_df.ix[6][0]) & set(king_df.ix[7][0]))

print('Net reach of King in May is: ', king_net_reach_may)
print('Net reach of King in June is: ', king_net_reach_june)
print('Net reach of King in July is: ', king_net_reach_july)

Net reach of King in May is:  33
Net reach of King in June is:  37
Net reach of King in July is:  39


In [13]:
# net reach for Supercell
supercell_df = user_growth_rates_df[user_growth_rates_df['company_name']=='Supercell']
supercell_df = pd.DataFrame(supercell_df.groupby(['month']).user_id.unique())
supercell_net_reach_may = 19
supercell_net_reach_june = supercell_net_reach_may + 17 - len(set(supercell_df.ix[5][0]) & set(supercell_df.ix[6][0]))
supercell_net_reach_july = supercell_net_reach_june + 18 - len(set(supercell_df.ix[6][0]) & set(supercell_df.ix[7][0]))

print('Net reach of Supercell in May is: ', supercell_net_reach_may)
print('Net reach of Supercell in June is: ', supercell_net_reach_june)
print('Net reach of Supercell in July is: ', supercell_net_reach_july)

Net reach of Supercell in May is:  19
Net reach of Supercell in June is:  20
Net reach of Supercell in July is:  21


In [14]:
# net reach for Zynga
zynga_df = user_growth_rates_df[user_growth_rates_df['company_name']=='Zynga']
zynga_df = pd.DataFrame(zynga_df.groupby(['month']).user_id.unique())
zynga_net_reach_may = 28
zynga_net_reach_june = zynga_net_reach_may + 30 - len(set(zynga_df.ix[5][0]) & set(zynga_df.ix[6][0]))
zynga_net_reach_july = zynga_net_reach_june + 29 - len(set(zynga_df.ix[6][0]) & set(zynga_df.ix[7][0]))

print('Net reach of Zynga in May is: ', zynga_net_reach_may)
print('Net reach of Zynga in June is: ', zynga_net_reach_june)
print('Net reach of Zynga in July is: ', zynga_net_reach_july)

Net reach of Zynga in May is:  28
Net reach of Zynga in June is:  31
Net reach of Zynga in July is:  34


#### c) What is the total time spent per month per user for Supercell, King and Zynga for July?

In [15]:
user_growth_rates_df['duration'] = (user_growth_rates_df['end_timestamp'] - user_growth_rates_df['start_timestamp']).astype('timedelta64[m]')
user_growth_rates_df.groupby(['company_name', 'month']).duration.sum()

company_name  month
Amazon        5         38167.0
              6         33819.0
              7         32548.0
Facebook      5        656900.0
              6        687815.0
              7        691026.0
Google        5        258387.0
              6        254084.0
              7        255790.0
King          5        158520.0
              6        132718.0
              7        154668.0
Lovefilm      5           978.0
              6          2136.0
              7           388.0
Supercell     5         46873.0
              6         42948.0
              7         49130.0
Walmart       5         13093.0
              6         10339.0
              7          8233.0
Zynga         5         54898.0
              6         58276.0
              7         56731.0
Name: duration, dtype: float64

In [16]:
king_time_july = 154668 / 60 / 36
supercell_time_july = 49130 / 60 / 18
zynga_time_july = 56731 / 60 / 29

print('Total time spent per month per user for Supercell, King and Zynga are: \
{:.2f}, {:.2f} and {:.2f} hours, respectively.'.format(supercell_time_july, king_time_july, zynga_time_july))

Total time spent per month per user for Supercell, King and Zynga are: 45.49, 71.61 and 32.60 hours, respectively.


#### d) Which of Supercell, King and Zynga had the largest average session duration in June?

In [17]:
user_growth_rates_df.groupby(['company_name', 'month']).duration.size()

company_name  month
Amazon        5         2975
              6         2920
              7         2912
Facebook      5        31020
              6        30733
              7        30996
Google        5         4593
              6         4575
              7         4599
King          5         2987
              6         2926
              7         2925
Lovefilm      5            6
              6            2
              7            6
Supercell     5         1820
              6         1768
              7         1868
Walmart       5          652
              6          625
              7          618
Zynga         5         3384
              6         3487
              7         3530
dtype: int64

In [18]:
king_average_duration_june = 132718 / 2926
supercell_average_duration_june = 42948 / 1768
zynga_average_duration_june = 58276 / 3487

print('Average session duration for Supercell, King and Zynga in June are: \
{:.2f}, {:.2f} and {:.2f} minutes, respectively.'.format(supercell_average_duration_june, \
king_average_duration_june, zynga_average_duration_june))
print('So King had the largest average session duration in June.')

Average session duration for Supercell, King and Zynga in June are: 24.29, 45.36 and 16.71 minutes, respectively.
So King had the largest average session duration in June.


#### e) What is the gender distribution of Supercell vs. King users?

In [19]:
user_growth_rates_df = pd.merge(user_growth_rates_df, demographics_df, on='user_id', how='left')
user_growth_rates_df

,device_id,end_timestamp,interaction_type,os,start_timestamp,subject_id,user_id,property,company_name,month,duration,age,gender,income_level
0,10212,2016-07-02 05:35:36,app,android,2016-07-02 05:29:24,5245226,2694,youtube,Google,7,6.0,53,1,4
1,10212,2016-07-19 13:33:20,app,android,2016-07-19 13:30:27,5245226,2694,youtube,Google,7,2.0,53,1,4
2,10212,2016-06-26 19:13:03,app,android,2016-06-26 19:10:29,5245226,2694,youtube,Google,6,2.0,53,1,4
3,10212,2016-05-18 02:03:57,app,android,2016-05-18 01:51:42,5245226,2694,youtube,Google,5,12.0,53,1,4
4,10212,2016-06-23 05:06:51,app,android,2016-06-23 05:02:43,5245226,2694,youtube,Google,6,4.0,53,1,4
5,10212,2016-06-02 13:42:29,app,android,2016-06-02 13:31:02,5245226,2694,youtube,Google,6,11.0,53,1,4
6,10212,2016-05-04 18:18:28,app,android,2016-05-04 17:38:00,5245226,2694,youtube,Google,5,40.0,53,1,4
7,10212,2016-05-27 14:34:32,app,android,2016-05-27 14:00:50,5245226,2694,youtube,Google,5,33.0,53,1,4
8,10212,2016-06-24 06:42:03,app,android,2016-06-24 06:16:09,5245226,2694,youtube,Google,6,25.0,53,1,4
9,10212,2016-05-14 10:18:08,app,android,2016-05-14 10:15:52,5245226,2694,youtube,Google,5,2.0,53,1,4


In [20]:
user_growth_rates_df.groupby(['company_name', 'gender']).user_id.nunique()

company_name  gender
Amazon        1         51
              2         43
Facebook      1         55
              2         45
Google        1         54
              2         45
King          1         17
              2         22
Lovefilm      1          5
              2          6
Supercell     1         14
              2          6
Walmart       1         39
              2         33
Zynga         1         18
              2         16
Name: user_id, dtype: int64

### Phase 2: Clash of Clans vs. Candy Crush Saga

#### a)  What’s the reach of Clash of Clans among Candy Crush Saga users?

In [21]:
clans_and_candy_df = user_growth_rates_df[user_growth_rates_df['property'].isin(['clash of clans', 'candy crush saga'])]
clans_and_candy_df.groupby(['property']).user_id.nunique()

property
candy crush saga    28
clash of clans      14
Name: user_id, dtype: int64

In [22]:
print('The reach of Clash of Clans among Candy Crush Saga users are 14% and 28%.')

The reach of Clash of Clans among Candy Crush Saga users are 14% and 28%.


#### b)  What’s the reach of Candy Crush Saga among users aged 35 or above? How does this compare to Clash of Clans?

In [23]:
clans_and_candy_larger_than_35_df = clans_and_candy_df[clans_and_candy_df['age'] >= 35]
clans_and_candy_larger_than_35_df.groupby(['property']).user_id.nunique()

property
candy crush saga    22
clash of clans      11
Name: user_id, dtype: int64

In [24]:
print('The reach of Clash of Clans among Candy Crush Saga users that aged 35 or above are 11% and 22%.')

The reach of Clash of Clans among Candy Crush Saga users that aged 35 or above are 11% and 22%.


#### c)  How many users play Candy Crush Saga on Tablets?

In [25]:
candy_crush_saga_df = clans_and_candy_df[clans_and_candy_df['property'].isin(['candy crush saga'])]
candy_crush_saga_devices_df = pd.merge(candy_crush_saga_df, devices_df, on='device_id', how='left')
candy_crush_saga_devices_df.groupby(['device_type']).user_id_x.nunique()

device_type
Smartphone    21
Tablet         7
Name: user_id_x, dtype: int64

In [26]:
print('There are 7 users playing Candy Crush Saga on Tablets.')

There are 7 users playing Candy Crush Saga on Tablets.


#### d)  What’s the deduplicated net reach of Clash of Clans and Candy Crush Saga combined? 

In [27]:
clans_and_candy_df.groupby(['property', 'month']).user_id.nunique()

property          month
candy crush saga  5        24
                  6        25
                  7        26
clash of clans    5        14
                  6        12
                  7        13
Name: user_id, dtype: int64

In [28]:
candy_df = clans_and_candy_df[clans_and_candy_df['property']=='candy crush saga']
candy_df = pd.DataFrame(candy_df.groupby(['month']).user_id.unique())
candy_deduplicated_net_reach = len(set(candy_df.ix[5][0]) & set(candy_df.ix[6][0])) + \
len(set(candy_df.ix[6][0]) & set(candy_df.ix[7][0]))

print(candy_deduplicated_net_reach)

47


In [29]:
clash_df = clans_and_candy_df[clans_and_candy_df['property']=='clash of clans']
clash_df = pd.DataFrame(clash_df.groupby(['month']).user_id.unique())
clash_deduplicated_net_reach = len(set(clash_df.ix[5][0]) & set(clash_df.ix[6][0])) + \
len(set(clash_df.ix[6][0]) & set(clash_df.ix[7][0]))

print(clash_deduplicated_net_reach)

24


In [30]:
print('There are {} deduplicated net reach of Clash of Clans and Candy Crush Saga combined.'.format(\
candy_deduplicated_net_reach+ clash_deduplicated_net_reach))

There are 71 deduplicated net reach of Clash of Clans and Candy Crush Saga combined.


### Phase 3: Advertising Channels

#### a)  What’s the reach of each of YouTube, Facebook, Amazon  and Walmart among King and Zynga gamers who are not Supercell gamers? 

In [31]:
supercell_user_df = user_growth_rates_df[user_growth_rates_df['company_name'].isin(['Supercell'])]
supercell_users = list(supercell_user_df['user_id'].unique())
non_supercell_user_df = user_growth_rates_df[~user_growth_rates_df['user_id'].isin(supercell_users)]

king_and_zynga_user_df = non_supercell_user_df[non_supercell_user_df['company_name'].isin(['King', 'Zynga'])]
king_and_zynga_user_users = list(king_and_zynga_user_df['user_id'].unique())
king_and_zynga_no_supercell_user_df = user_growth_rates_df[user_growth_rates_df['user_id'].isin(king_and_zynga_user_users)]
king_and_zynga_no_supercell_user_df.groupby('company_name').user_id.nunique()
# king_and_zynga_user_df = non_supercell_user_df[non_supercell_user_df['company_name'].isin(['King', 'Zynga'])]
# king_and_zynga_user_df.groupby(['company_name']).user_id.nunique()

company_name
Amazon      55
Facebook    59
Google      58
King        34
Lovefilm     8
Walmart     44
Zynga       33
Name: user_id, dtype: int64

In [32]:
print('The reach of YouTube, Facebook, Amazon and Walmart among King and Zynga gamers who are not Supercell gamers are: {}%, {}%, {}% and {}%.'\
.format(58, 59, 55, 44))

The reach of YouTube, Facebook, Amazon and Walmart among King and Zynga gamers who are not Supercell gamers are: 58%, 59%, 55% and 44%.


#### b)  What is Facebook’s average number of sessions per active user per month among Clash of Clan players? And what is it among Candy Crush Saga users? 

In [33]:
clash_user_session_df = user_growth_rates_df[user_growth_rates_df['property'].isin(['clash of clans'])]
clash_users = list(clash_user_session_df['user_id'].unique())
clash_user_df = user_growth_rates_df[user_growth_rates_df['user_id'].isin(clash_users)]

candy_user_session_df = user_growth_rates_df[user_growth_rates_df['property'].isin(['candy crush saga'])]
candy_users = list(candy_user_session_df['user_id'].unique())
candy_user_df = user_growth_rates_df[user_growth_rates_df['user_id'].isin(candy_users)]

In [34]:
clash_facebook_sessions = len(clash_user_df[clash_user_df['company_name']=='Facebook'])
clash_facebook_users = clash_user_df[clash_user_df['company_name']=='Facebook'].user_id.nunique()
clash_average_facebook_number = clash_facebook_sessions / clash_facebook_users / 3

candy_facebook_sessions = len(candy_user_df[candy_user_df['company_name']=='Facebook'])
candy_facebook_users = candy_user_df[candy_user_df['company_name']=='Facebook'].user_id.nunique()
candy_average_facebook_number = candy_facebook_sessions / candy_facebook_users / 3

print("Facebook’s average number of sessions per active user per month among Clash of Clan players and Candy Crush Saga users are: {:.1f} and {:.1f}, respectlvely."\
.format(clash_average_facebook_number, candy_average_facebook_number))

Facebook’s average number of sessions per active user per month among Clash of Clan players and Candy Crush Saga users are: 233.3 and 362.6, respectlvely.
